<a href="https://colab.research.google.com/github/choi-yh/DataStructure/blob/master/2_6_Image_Viewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5.QtGui import *
from PyQt5.QtCore import *

# CircleLinkedList
class Node:
    def __init__(self, item=None):
        self.item = item
        self.link = None


class CircleLinkedList:
    def __init__(self):
        self.root = Node()
        self.tail = self.root
        self.current = self.root

    def append(self, item):
        newNode = Node(item)
        if self.root.item == None:
            self.root = newNode
            self.tail = newNode
            newNode.link = self.root
        else:
            self.tail.link = newNode
            newNode.link = self.root
            self.tail = newNode

    def setCurrent(self, item):
        curNode = self.root
        while curNode.item != item:
            curNode = curNode.link
        self.current = curNode

    def moveNext(self):
        self.current = self.current.link
        print("현재 위치는 ", self.current.item, "입니다.")

    def insert(self, item):
        newNode = Node(item)
        newNode.link = self.current.link
        self.current.link = newNode
        # self.current가 tail일 경우 새 노드를 tail로 재지정
        if self.current == self.tail:
            self.tail = newNode

    def delete(self, item):
        delYN = False  # 지울 노드가 리스트에 있는지 확인
        curNode = self.root
        # root 노드를 지울 경우 root와 tail 갱신
        if self.root.item == item:
            self.root = self.root.link
            self.tail = self.root
            delYN = True
        # tail 노드를 지울경우 tail 갱신
        else:
            while curNode.link != self.root:
                preNode = curNode
                curNode = curNode.link
                if curNode.item == item:
                    preNode.link = curNode.link
                    if curNode == self.tail:
                        self.tail = preNode
                    delYN = True
        if delYN == False:
            print("해당 아이템이 리스트에 없습니다.")

    def listSize(self):
        curNode = self.root
        cnt = 1
        while curNode.link != self.root:
            curNode = curNode.link
            cnt += 1
        return cnt

    def print(self):
        curNode = self.root
        while curNode.link != self.root:
            print(curNode.item, end=' - ')
            curNode = curNode.link
        print(curNode.item)


# PyQt5
form_class = uic.loadUiType("C:\\Users\\yhcho\\Anaconda3\\Lib\\site-packages\\PyQt5\\uic\\Image_Window.ui")[0]


class MyWindow(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.qPixmapVar = QPixmap()

        self.actionOpen.triggered.connect(self.openDir)
        self.actionClose.triggered.connect(qApp.quit) # quitAction

        self.pushButton.clicked.connect(self.moveNext)  # moveNext
        self.pushButton_2.clicked.connect(self.slideShow)  # Slide Show

    def openDir(self):
        dname = str(QFileDialog.getExistingDirectory(self, "Select Directory"))  # dir 폴더 불러오기
        file_list = os.listdir(dname)

        # CircleLinkedList에 삽입
        self.imgList = CircleLinkedList()
        for img in file_list:
            self.imgList.append(dname + '/' + img)
        self.imgList.current = self.imgList.root
        self.imgList.print()
        print(self.imgList.current.item)

        # 이미지 띄우기
        self.qPixmapVar.load(self.imgList.current.item)
        self.qPixmapVar = self.qPixmapVar.scaled(800, 600, aspectRatioMode=True)
        self.label.setPixmap(self.qPixmapVar)

    def moveNext(self):
        self.imgList.moveNext()
        self.qPixmapVar.load(self.imgList.current.item)
        self.qPixmapVar = self.qPixmapVar.scaled(800, 600, aspectRatioMode=True)
        self.label.setPixmap(self.qPixmapVar)

    def slideShow(self):
        # 이동 타이머 설정
        self.timer = QTimer(self)
        self.timer.setInterval(3000)
        self.timer.timeout.connect(self.moveNext)
        self.timer.start()


if __name__ == "__main__":
    import sys
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()